# Pinecone DB (프리티어 사용)

- 사용방법 참조
- https://app.pinecone.io/organizations/-OO6qXaTMjHv0vo82OHB/projects/47c50289-22ec-41ee-98a1-1de664766c8d/indexes

In [ ]:
from dotenv import load_dotenv
import os
from pinecone import Pinecone, ServerlessSpec

load_dotenv()
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")

pc = Pinecone(api_key=PINECONE_API_KEY)
index_name = "reviewtest"

# Pinecone 링크(위에 적어놓은 링크)로 접속해서 Create index 항목 참조
pc.create_index(
    name=index_name,
    dimension=384,
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws",        # vectordb를 올려놓을 cloud 선정 | awd: 아마존에서 제공
        region="us-east-1"
    )
)

{
    "name": "reviewtest",
    "metric": "cosine",
    "host": "reviewtest-kmvd9ik.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 384,
    "deletion_protection": "disabled",
    "tags": null
}

In [3]:
reviews = [
    {"id": "1", "text": "이 노트북은 부팅 속도도 빠르고 디자인도 세련됐어요.", "sentiment": "positive"},
    {"id": "2", "text": "팬 소음이 심하고 발열도 심해서 사용이 불편합니다.", "sentiment": "negative"},
    {"id": "3", "text": "가성비가 뛰어나고 사양도 괜찮아서 만족스럽습니다.", "sentiment": "positive"},
    {"id": "4", "text": "배터리가 금방 닳아서 외출 시 불안해요.", "sentiment": "negative"},
    {"id": "5", "text": "화면 밝기나 색감이 아주 좋고 눈이 편안해요.", "sentiment": "positive"},
    {"id": "6", "text": "윈도우 설치부터 오류가 많아 고생했어요.", "sentiment": "negative"},
    {"id": "7", "text": "타이핑 감이 좋아서 장시간 작업해도 손목이 편해요.", "sentiment": "positive"},
    {"id": "8", "text": "생각보다 무겁고 휴대성이 떨어지네요.", "sentiment": "negative"},
    {"id": "9", "text": "AS 속도도 빠르고 친절해서 신뢰가 갑니다.", "sentiment": "positive"},
    {"id": "10", "text": "처음부터 키보드 일부가 안 눌리는 문제가 있었어요.", "sentiment": "negative"},
    {"id": "11", "text": "영상 편집도 무리 없이 돌아가서 만족해요.", "sentiment": "positive"},
    {"id": "12", "text": "충전기가 너무 크고 무거워서 불편해요.", "sentiment": "negative"},
    {"id": "13", "text": "터치패드 반응이 빠르고 정밀해서 좋습니다.", "sentiment": "positive"},
    {"id": "14", "text": "포장 상태가 엉망이고 외관에 흠집도 있었어요.", "sentiment": "negative"},
    {"id": "15", "text": "화면 회전과 터치 기능까지 있어서 활용도가 높아요.", "sentiment": "positive"},
    {"id": "16", "text": "사운드가 너무 작고 음질도 별로예요.", "sentiment": "negative"},
    {"id": "17", "text": "프로그램 실행 속도가 빨라서 업무용으로 최고예요.", "sentiment": "positive"},
    {"id": "18", "text": "USB 포트가 너무 적어서 허브 없이는 불편해요.", "sentiment": "negative"},
    {"id": "19", "text": "디스플레이 해상도가 높아서 눈이 정말 편안해요.", "sentiment": "positive"},
    {"id": "20", "text": "지문 인식이 잘 안 되고 보안 기능도 미흡해요.", "sentiment": "negative"}
]

In [ ]:
# Pinecone index 및 임베딩 모델 로드
from dotenv import load_dotenv
import os
from pinecone import Pinecone, ServerlessSpec
from sentence_transformers import SentenceTransformer

load_dotenv()
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")

index_name = "reviewtest"
pc = Pinecone(api_key=PINECONE_API_KEY)
idx = pc.Index(index_name)

model = SentenceTransformer('all-MiniLM-L6-v2')

In [5]:
# index에 리뷰 데이터 추가
for review in reviews:
    review_embed = model.encode(review['text']).tolist()
    idx.upsert([
        (   # 튜플 형태
            review['id'],       # str 형태의 id
            review_embed,       # 임베딩벡터
            {   # dictionary 형태
                'text': review['text'],
                'sentiment': review['sentiment']
            }
        )
    ])

### 리뷰 데이터 조회

In [6]:
query_text = "이 노트북의 성능은 괜찮은가요?"
query_embed = model.encode(query_text).tolist()

results = idx.query(
    vector=query_embed,
    top_k=3,
    include_metadata=True
)

results

{'matches': [{'id': '1',
              'metadata': {'sentiment': 'positive',
                           'text': '이 노트북은 부팅 속도도 빠르고 디자인도 세련됐어요.'},
              'score': 0.727021098,
              'values': []},
             {'id': '4',
              'metadata': {'sentiment': 'negative',
                           'text': '배터리가 금방 닳아서 외출 시 불안해요.'},
              'score': 0.700294375,
              'values': []},
             {'id': '8',
              'metadata': {'sentiment': 'negative',
                           'text': '생각보다 무겁고 휴대성이 떨어지네요.'},
              'score': 0.675620735,
              'values': []}],
 'namespace': '',
 'usage': {'read_units': 6}}

In [7]:
for review in results['matches']:
    metadata = review['metadata']
    print(metadata['text'], f'(감성: {metadata["sentiment"]})')

이 노트북은 부팅 속도도 빠르고 디자인도 세련됐어요. (감성: positive)
배터리가 금방 닳아서 외출 시 불안해요. (감성: negative)
생각보다 무겁고 휴대성이 떨어지네요. (감성: negative)


### 리뷰 감성 분석

In [8]:
query_text = "배터리도 빨리 닳고 벽돌보다 무거워요. 비추천"
query_embed = model.encode(query_text).tolist()

results = idx.query(
    vector=query_embed,
    top_k=5,
    include_metadata=True
)

results

{'matches': [{'id': '10',
              'metadata': {'sentiment': 'negative',
                           'text': '처음부터 키보드 일부가 안 눌리는 문제가 있었어요.'},
              'score': 0.820163727,
              'values': []},
             {'id': '8',
              'metadata': {'sentiment': 'negative',
                           'text': '생각보다 무겁고 휴대성이 떨어지네요.'},
              'score': 0.819359124,
              'values': []},
             {'id': '11',
              'metadata': {'sentiment': 'positive',
                           'text': '영상 편집도 무리 없이 돌아가서 만족해요.'},
              'score': 0.807165384,
              'values': []},
             {'id': '12',
              'metadata': {'sentiment': 'negative',
                           'text': '충전기가 너무 크고 무거워서 불편해요.'},
              'score': 0.802863061,
              'values': []},
             {'id': '20',
              'metadata': {'sentiment': 'negative',
                           'text': '지문 인식이 잘 안 되고 보안 기능도 미흡해요.'},
              'score': 0.78840953

In [10]:
sentiment_counts = {
    "positive": 0,
    "negative": 0
}

for review in results['matches']:
    sentiment = review['metadata']["sentiment"]     # 결과: "positive" or "negative"
    sentiment_counts[sentiment] += 1

print(f"리뷰 분석 결과: {'positive' if sentiment_counts['positive'] > sentiment_counts['negative'] else 'negative'}")

리뷰 분석 결과: negative
